In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import concurrent.futures
import string
import os
import time
import pandas as pd







In [2]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AUtomationControlled")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")


In [130]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options) # Esto lo defino dentro de las funciones para que se inicialice
driver.quit()
url = "https://www.thgeyer.com/lab/shop/labware"

TH GEYER: ORGANIZACION PAGINA (16/12/2024)

CATEGORIA --> SUBCATEGORIA --> SUB-SUBCATEGORIA --> LISTA DE PRODUCTOS --> IMAGEN Y DESCRIPCIÓN DE PRODUCTO --> LISTA CON DATOS

In [4]:
def scrapeo_categorias(url):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    data = pd.DataFrame(columns=["categoria", "categoriaLink"])

    try:
        driver.get(url)
        time.sleep(3)

        links = driver.find_elements(By.XPATH,"//a[contains(@href,'labware')]")

        for index, link in enumerate(links):
            href = link.get_attribute("href")
            text = link.text.strip()
            if 'labware' in href and text and text.isupper() and (text != 'LABWARE'):
                data.loc[len(data)] = [text,href] # Categoria, CategoriaLink

    except Exception as e:
        print(f"Ha ocurrido un error: {e}")
    driver.quit()

    return data

In [5]:
data = scrapeo_categorias(url)
data

,categoria,categoriaLink
0,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...
1,"OCCUPATIONAL SAFETY, SECURITY",https://www.thgeyer.com/lab/shop/labware/occup...
2,ANALYTICAL MEASUREMENT AND TESTING,https://www.thgeyer.com/lab/shop/labware/analy...
3,"STIRRING, SHAKING, MIXING",https://www.thgeyer.com/lab/shop/labware/stirr...
4,SAMPLE PREPARATION,https://www.thgeyer.com/lab/shop/labware/sampl...
5,"DISTILLATION, SEPARATION, FILTRATION",https://www.thgeyer.com/lab/shop/labware/disti...
6,HEATING AND COOLING TECHNOLOGY,https://www.thgeyer.com/lab/shop/labware/heati...
7,LIQUID HANDLING,https://www.thgeyer.com/lab/shop/labware/liqui...
8,"VACUUM TECHNOLOGY, DRYING, DRY STORAGE",https://www.thgeyer.com/lab/shop/labware/vacuu...
9,OPTICAL INSTRUMENTS AND MICROSCOPES,https://www.thgeyer.com/lab/shop/labware/optic...


In [6]:
# comprobacion
for index, row in data.iterrows():
    print(f"Categoria: {row['categoria']} || Link: {row['categoriaLink']}")


Categoria: GENERAL LABORATORY CONSUMABLES || Link: https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: OCCUPATIONAL SAFETY, SECURITY || Link: https://www.thgeyer.com/lab/shop/labware/occupational-safety-security;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: ANALYTICAL MEASUREMENT AND TESTING || Link: https://www.thgeyer.com/lab/shop/labware/analytical-measurement-and-testing;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: STIRRING, SHAKING, MIXING || Link: https://www.thgeyer.com/lab/shop/labware/stirring-shaking-mixing;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: SAMPLE PREPARATION || Link: https://www.thgeyer.com/lab/shop/labware/sample-preparation;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: DISTILLATION, SEPARATION, FILTRATION || Link: https://www.thgeyer.com/lab/shop/labware/distillation-separation-filtration;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Categoria: HEATING 

SUBCATEGORIAS

In [7]:
def scrapeo_subcategorias(url):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    mapa_subcategorias = {} # nombre_subcategoria --> Link 
    try:
        driver.get(url)
        time.sleep(3)

        links = driver.find_elements(By.XPATH, "//a[@href]")
        for link in links:
            href = link.get_attribute("href")
            text = link.text.strip()

            if 'labware' in href and text and text.isupper() and (text != "LABWARE"):
                mapa_subcategorias[text] = href
    except Exception as e:
        print(f"Ha ocurrido la excepcion {e}")
    driver.quit()
    return mapa_subcategorias

In [8]:
# Ahora creo una coloumna nueva en el dataframe (data) --> la columna va a tener un mapa de las subcategorias y sus links
data["subcategoria_1"] = None

for index, row in data.iterrows():
    categoria_link = row["categoriaLink"]
    subcategorias = scrapeo_subcategorias(categoria_link)
    data.at[index,'subcategoria_1'] = subcategorias




In [9]:
# comprobacion
for index, row in data.iterrows():
    print(f"Categoria: {row['categoria']} || {row['subcategoria_1'].keys()}")

Categoria: GENERAL LABORATORY CONSUMABLES || dict_keys(['VESSELS', 'RACKS', 'TOOLS', 'HEATING', 'TUBING', 'GENERAL LABORATORY AIDS', 'CONSUMER GOODS'])
Categoria: OCCUPATIONAL SAFETY, SECURITY || dict_keys(['EYE PROTECTION', 'BREATHING PROTECTION', 'EAR PROTECTION', 'GLOVES', 'SKIN PROTECTION', 'PROTECTIVE CLOTHING', 'SAFETY DEVICES', 'IDENTIFICATION OF HAZARDS', 'FIRST-AID', 'WASTE DISPOSAL', 'SAFETY CONTAINERS', 'TRANSPORT', 'FURNITURE'])
Categoria: ANALYTICAL MEASUREMENT AND TESTING || dict_keys(['PH-MEASUREMENT', 'CONDUCTIVITY MEASUREMENT', 'OXYGEN MEASUREMENT', 'MULTIPARAMETER MEASUREMENT', 'BALANCES', 'THERMOMETERS', 'TIMING', 'CLIMATE MEASUREMENT', 'DENSITY DETERMINATION', 'VISCOSIMETERS', 'MATERIAL TESTING', 'ANALYTICAL INSTRUMENTS AND SYSTEMS'])
Categoria: STIRRING, SHAKING, MIXING || dict_keys(['MAGNETIC STIRRERS / HOTPLATES', 'OVERHEAD STIRRERS', 'SHAKERS AND MIXERS'])
Categoria: SAMPLE PREPARATION || dict_keys(['SAMPLING', 'GRINDING', 'SIEVING', 'LABORATORY PRESSES', 'MIXIN

SUBCATEGORIAS DE NIVEL 2 --> CADA SUBCATEGORIA TIENE SUS PROPIAS SUBCATEGORIAS

Como tarda mucho, voy a utilizar la libreria "concurrent.futures" para poder crear hilos. 

Como funciona la logica:

Categoria --> Subcategoria (mapa) --> lista URLS --> 10 hilos, cada uno se dedica a una parte porcentual de las urls

In [ ]:
# POR HACER


In [27]:
data["subcategoria_2"] = None
for index,row in data.iterrows():
    mapa_subcategoria = row["subcategoria_1"]  #  {Subcategoria,link}
    mapa_subcategoria_y_subSubcategoria = {}
    for key in mapa_subcategoria.keys():
        link = str(mapa_subcategoria[key])
        print(link)
        subcategorias_nivel_2 = scrapeo_subcategorias(link)    # {sub-subcategoria,link_sub_subcategoria}
        mapa_subcategoria_y_subSubcategoria[key] = subcategorias_nivel_2  # {Subcategoria, {sub-subcategoria, link_sub_subcategoria}}
    
    data.at[index,"subcategoria_2"] =  mapa_subcategoria_y_subSubcategoria 


https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/racks;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/tools;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/heating;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/tubing;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/general-laboratory-aids;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/consumer-goods;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
https://www.thgeyer.com/lab/shop/labware/occupational-safety-security/eye-protection;jsessionid=2B6CACC59AC487D709

In [33]:

data["subcategoria_2"][0]

{'VESSELS': {'BEAKERS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/beakers;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'MEASURING JUGS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/measuring-jugs;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'FLASKS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/flasks;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'TEST TUBES': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/test-tubes;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'STOPPERS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/stoppers;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'CONTAINERS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/containers;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845',
  'BASKETS': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-c

AHORA EXPORTAR TODA LA ESTRUCTURA A FORMATO CSV --> TODOS LOS DATOS MENOS LA PARTE DE LOS PRODUCTOS

In [96]:
dataCSV = pd.DataFrame(columns=["categoria","categoriaLink","subcategoria","subcataegoriaLink","subSubCategoria","subSubCategoriaLink"])
# Lista para almacenar las filas temporalmente
rows = []
for index,row in data.iterrows():
    categoria = row["categoria"]
    categoriaLink = row["categoriaLink"]
    mapa1 = row["subcategoria_1"]
    mapa2 = row["subcategoria_2"]
    mapa3 = {}
    #print(f"Subcategorias de la Categoria '{categoria}': \n")
    for i,key in enumerate(mapa1.keys()):
        nombre_subcategoria = key
        link_subcategoria = mapa1[key]
        #print(f"- {key} | {link_subcategoria} \n")
        mapa3 = mapa2[key]
        for key2 in mapa3.keys():
            nombre_subsubcategoria = key2
            link_subsubcategoria = mapa3[key2]
            #print(f"\t * {nombre_subsubcategoria} | {link_subsubcategoria} \n")

            rows.append({
                "categoria": categoria,
                "categoriaLink": categoriaLink,
                "subcategoria": nombre_subcategoria,
                "subcategoriaLink": link_subcategoria,
                "subSubCategoria": nombre_subsubcategoria,
                "subSubCategoriaLink": link_subsubcategoria
            })
        
dataCSV = pd.DataFrame(rows)

# Mostrar el DataFrame resultante
len(dataCSV)
dataCSV.to_csv("listaUrlsConProductos.csv", index=False)




KeyError: 'subcategoria_1'

AHORA CARGO LOS URLS DONDE ESTAN LOS PRODUCTOS DESDE EL CSV PARA NO TENER QUE EJECUTAR TODO

En caso de que la pagina cambie o suban nuevos productos --> Volver a ejecutar lo anterior, comparar con "listaUrlsConProductos.csv" y despues añadir los elementos que no coincidan

PRODUCTOS

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import concurrent.futures
import string
import os
import time
import pandas as pd


In [25]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AUtomationControlled")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
service = Service(ChromeDriverManager().install())

In [ ]:
# cargar los datos del csv

data = pd.read_csv("listaUrlsConProductos.csv")
data.head()

,categoria,categoriaLink,subcategoria,subcategoriaLink,subSubCategoria,subSubCategoriaLink
0,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...,VESSELS,https://www.thgeyer.com/lab/shop/labware/gener...,BEAKERS,https://www.thgeyer.com/lab/shop/labware/gener...
1,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...,VESSELS,https://www.thgeyer.com/lab/shop/labware/gener...,MEASURING JUGS,https://www.thgeyer.com/lab/shop/labware/gener...
2,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...,VESSELS,https://www.thgeyer.com/lab/shop/labware/gener...,FLASKS,https://www.thgeyer.com/lab/shop/labware/gener...
3,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...,VESSELS,https://www.thgeyer.com/lab/shop/labware/gener...,TEST TUBES,https://www.thgeyer.com/lab/shop/labware/gener...
4,GENERAL LABORATORY CONSUMABLES,https://www.thgeyer.com/lab/shop/labware/gener...,VESSELS,https://www.thgeyer.com/lab/shop/labware/gener...,STOPPERS,https://www.thgeyer.com/lab/shop/labware/gener...


In [18]:
data["subSubCategoriaLink"][data["subSubCategoria"]== "HOTPLATES"].head(1)

52    https://www.thgeyer.com/lab/shop/labware/gener...
Name: subSubCategoriaLink, dtype: object

In [66]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def scrape_product_info(url):
    # Configura el driver
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Abre la página
    driver.get(url)

    # Esperar a que la página se cargue completamente (ajusta según la necesidad)
    time.sleep(3)

    # Lista para almacenar los datos de los productos
    products_data = []

    # Extracción de productos generales
    products = driver.find_elements(By.CLASS_NAME, "search_gruppenElement")
    for product in products:
        try:
            # Extraer nombre del producto
            product_name = product.find_element(By.CLASS_NAME, "segment_headline").text
            
            # Extraer marca (texto dentro de <b>)
            brand = product.find_element(By.XPATH, ".//div[@class='col-xs-4 col-md-2']/b").text
            
            # Extraer imagen (src del <img>)
            image_url = product.find_element(By.CLASS_NAME, "img-responsive").get_attribute("src")
            
            # Extraer descripción (texto dentro del <p>)
            description_text = product.find_element(By.CSS_SELECTOR, ".overflow-text-dots p").text

            # Agregar datos generales del producto
            product_info = {
                "product_name": product_name,
                "brand": brand,
                "image_url": image_url,
                "description": description_text,
                "url": url  # URL de la página para referencia
            }

            products_data.append(product_info)

            # Imprimir datos (opcional)
            print(f"Producto General: {product_info}")
            print("-" * 50)

        except Exception as e:
            print(f"Error al extraer datos de un producto general: {e}")

    # Extracción de datos de la tabla específica
    try:
        # Verificar si la tabla está presente en la página
        if len(product.find_element(By.TAG, "table")) > 0:
            # Localizar la tabla por su ID
            table = product.find_element(By.TAG, "table")
            

            # Encontrar el encabezado de la tabla (nombres de las columnas)
            headers = [header.text.strip() for header in table.find_elements(By.XPATH, ".//thead/tr/th")]

            # Encontrar todas las filas dentro del tbody de la tabla
            rows = table.find_elements(By.XPATH, ".//tbody/tr")

            for row in rows:
                try:
                    # Extraer todas las celdas de la fila
                    cells = row.find_elements(By.XPATH, "./td")

                    # Crear un mapa dinámico entre encabezados y valores
                    row_data = {headers[i]: cells[i].text.strip() for i in range(len(cells)) if i < len(headers)}

                    # Agregar los datos mapeados a un producto
                    table_product_info = {
                        "product_type": "table_row",  # Indicador del origen de este producto
                        "table_data": row_data  # Datos de la fila como un diccionario
                    }

                    # Agregar el producto de la tabla al listado general
                    products_data.append(table_product_info)

                    # Imprimir datos (opcional)
                    print(f"Producto de Tabla: {table_product_info}")
                    print("-" * 50)

                except Exception as e:
                    print(f"Error al extraer datos de una fila de la tabla: {e}")
        else:
            print("La tabla con ID 'tablegruppe1489' no está presente en esta página.")

    except Exception as e:
        print(f"Error al procesar la tabla: {e}")

    # Cerrar el navegador
    driver.quit()

    # Retornar la lista completa de productos
    return products_data

In [60]:


# def scrape_product_info(url):
#     # Configura el driver
#     driver = webdriver.Chrome(service=service, options=chrome_options)

#     # Abre la página
#     driver.get(url)

#     # Esperar a que la página se cargue (ajusta según la necesidad)
#     time.sleep(3)

#     # Lista para almacenar los productos
#     products_data = []

#     # Encontrar todos los productos en la página
#     products = driver.find_elements(By.CLASS_NAME, "search_gruppenElement")

#     # Iterar sobre cada producto y extraer la información
#     for product in products:
#         try:
#             # Extraer nombre del producto
#             product_name = product.find_element(By.CLASS_NAME, "segment_headline").text
            
#             # Extraer marca (texto dentro de <b>)
#             brand = product.find_element(By.XPATH, ".//div[@class='col-xs-4 col-md-2']/b").text
            
#             # Extraer imagen (src del <img>)
#             image_url = product.find_element(By.CLASS_NAME, "img-responsive").get_attribute("src")
            
#             # Extraer descripción (texto dentro del <p>)
#             description_text = product.find_element(By.CSS_SELECTOR, ".overflow-text-dots p").text
          
#             # Crear un diccionario con los datos del producto
#             product_info = {
#                 "product_name": product_name,
#                 "brand": brand,
#                 "image_url": image_url,
#                 "description": description_text,
#                 "url": url  # Añadir la URL de la página a cada producto
#             }

#             # Añadir el diccionario a la lista
#             products_data.append(product_info)

#             # Imprimir los datos del producto (opcional)
#             print(f"Nombre del Producto: {product_name}")
#             print(f"Marca: {brand}")
#             print(f"Imagen: {image_url}")
#             print(f"Descripción: {description_text}")
#             print(f"URL: {url}")
#             print("-" * 50)  # Separador entre productos
            
#         except Exception as e:
#             print(f"Error al extraer datos de un producto: {e}")

#     # Cerrar el navegador
#     driver.quit()

#     # Retornar la lista de productos
#     return products_data


In [67]:
link = data["subSubCategoriaLink"][1]
print(link)
#print(conseguir_productos(link))
product_list= scrape_product_info(link)#   funciona

for product in product_list:
    print(product)
    print("-" * 50)

https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/measuring-jugs;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845
Producto General: {'product_name': 'MEASURING BEAKERS WITH CLOSED HANDLE, PP', 'brand': 'LABSOLUTE®', 'image_url': 'https://www.thgeyer.com/lab/shop/pic/150x150/owweb/7693022.jpg;jsessionid=43BDCE65FE769DBCF4139615FF290355', 'description': 'LABSOLUTE® measuring and griffin beakers are made of chemical-resistant, transparent PP and are therefore particularly lightweight. They feature a clearly legible printed scale and a functional spout and are approved for food use and autoclavable up to 121 °C. Production is compliant with ISO 7056.\n...', 'url': 'https://www.thgeyer.com/lab/shop/labware/general-laboratory-consumables/vessels/measuring-jugs;jsessionid=2B6CACC59AC487D709E3DAAC47DFD845'}
--------------------------------------------------
Producto General: {'product_name': 'GRIFFIN BEAKERS (MEASURING BEAKERS WITHOUT HANDLE), PP', 'brand': 'LABSOLUTE®